<a href="https://colab.research.google.com/github/yodavo/moja-global-analysis/blob/main/2021-12-16_Task2-Holdrigde/lifezones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install GDAL and Geopandas
!apt install gdal-bin python-gdal python3-gdal -q
!apt install python3-rtree -q
!pip install geopandas -q
!pip install descartes -q

# Folium 
!pip install folium -q

# for rasters
!pip install Rasterio -q
!pip install rasterstats -q
!pip install georasters -q

# Install Pysal - Spatial Statistics
!pip install pysal -q
!pip install splot -q

In [2]:
import os, zipfile, glob
import rasterio,  gdal
import geopandas as gpd
import rasterstats
import numpy as np
from rasterio.plot import show
from rasterio.plot import show_hist
from matplotlib import pyplot as plt
ith zipfile.ZipFile('clip_tiffWorldclim[Colombia-Belize].zip', 'r') as temp:
#   temp.extractall("rasters")

# # path to the folder
# files = 
import warnings

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


# **Clipping**

In [3]:
# creating folder to store information
%mkdir clip_shapefiles
%mkdir outcomes
# %mkdir geodataframes

In [4]:
# Function to clip rasters
def clip_raster(list_raster, inshp, wordclim_dataset):
  ''' Get a clipped raster from the Worldclim data for specified countries:
  - list_raster : geotiff fetched from Worldclim
  - inshp : path clipped shapefile with the name of the country
  - wordclim_dataset: name of the dataset for outraster file name '''
  for i in list_raster:
    name, raster = i
    outraster = '/content/outcomes/' + str(wordclim_dataset) + '_' + str.upper(name) + '.tif'
    dsClip = gdal.Warp(outraster, raster, cutlineDSName=inshp, cropToCutline=True, dstNodata=np.nan)
    
    # visualization
    # plt.imshow(dsClip.GetRasterBand(1).ReadAsArray())
    # plt.colorbar()
    # plt.title(str.capitalize(wordclim_dataset) + ' - ' + str.upper(name))

# **Boundaries**

In [5]:
# importing the data
world_df = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# subseting
world_df = world_df[['continent','name', 'iso_a3','geometry']]

# croppping boundaries
colombia_bd = world_df.loc[world_df.name == 'Colombia', ].reset_index(drop=True)
colombia_bd.to_file("/content/clip_shapefiles/colombia.shp")
belize_bd = world_df.loc[world_df.name == 'Belize', ].reset_index(drop=True)
belize_bd.to_file("/content/clip_shapefiles/belize.shp")

# path to shapefiles
colombia = '/content/clip_shapefiles/colombia.shp'
belize = '/content/clip_shapefiles/belize.shp'

# **Fetching Elevation 2.5m**

In [7]:
# fetching elevation geotif form worldclim 2.5m
! wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_2.5m_elev.zip -q --show-progress
with zipfile.ZipFile('wc2.1_2.5m_elev.zip', 'r') as tempm:
  tempm.extractall("temp")

# path to the folder
files = glob.glob(os.path.join('/content/temp', '*tif'))

####################### tiff file #################################
# Rename and arrange the information
name = files[0][files[0].rfind('/')+1 : files[0].rfind('.')]

# merging all the metadata
tiff_dem2_5 = [(name, gdal.Open(files[0]))]

############################ Clippping maps #################################
# Colombia
clip_raster(tiff_dem2_5, colombia, 'Colombia')

# Belize
clip_raster(tiff_dem2_5, belize, 'Belize')

##################### Removing Cache ################################
%rm "wc2.1_2.5m_elev.zip"
%rm -r "temp"

wc2.1_2.5m_elev.zip 100%[===================>]  16.75M  25.2MB/s    in 0.7s    


# **Fetching Elevation 30s**

In [8]:
# fetching elevation geotif form worldclim 30s
! wget https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_elev.zip -q --show-progress
with zipfile.ZipFile('wc2.1_30s_elev.zip', 'r') as tempm:
  tempm.extractall("temp")

# path to the folder
files = glob.glob(os.path.join('/content/temp', '*tif'))

####################### tiff file #################################
# Rename and arrange the information
name = files[0][files[0].rfind('/')+1 : files[0].rfind('.')]

# merging all the metadata
tiff_dem30 = [(name, gdal.Open(files[0]))]

############################ Clippping maps #################################
# Colombia
clip_raster(tiff_dem30, colombia, 'Colombia')

# Belize
clip_raster(tiff_dem30, belize, 'Belize')

##################### Removing Cache ################################
%rm "wc2.1_30s_elev.zip"
%rm -r "temp"


wc2.1_30s_elev.zip  100%[===================>] 322.52M  36.0MB/s    in 9.1s    


# **Elevation list** `elev_colombia` - `elev_belize`

In [23]:
############### Elevation information ################################
# path to the folder
eleva = glob.glob(os.path.join('/content/outcomes', '*tif'))

###### list of tiff files ######
# Rename and arrange the information
eleva_names = [i[i.rfind('/')+1:-4] for i in eleva]
eleva_names.sort()

##################### elevations #########################################
# Separate datasets (this is pretty redundant)
elev_Belize, elev_Colombia = [i for i in eleva_names if i.startswith('Belize')], [i for i in eleva_names if i.startswith('Colombia')]

# reading the data
pathe, exte = '/content/outcomes/', '.tif'

# importing and handling
elev_belize = []
for i in elev_Belize:
  name = i
  ds = rasterio.open(pathe + i + exte, 'r')
  data = ds.read()
  max = np.nanmax(data)
  min = np.nanmin(data)
  elev_belize.append((name, ds, data, min, max))

elev_colombia = []
for i in elev_Colombia:
  name = i
  ds = rasterio.open(pathe + i + exte, 'r')
  data = ds.read()
  max = np.nanmax(data)
  min = np.nanmin(data)
  elev_colombia.append((name, ds, data, min, max))

# **Temperature** `amt_colombia` - `amt_belize`  **and Precipitation list** `tpa_colombia` - `tap_belize` 

- **AMT** - BIO2 = Mean Diurnal Range (Mean of monthly (max temp - min temp))
- **TPA** - BIO12 = Annual Precipitation



In [39]:
#########################################################################
#################### Manual retrival ####################################
#########################################################################

# get the information in the zip file
with zipfile.ZipFile('clip_tiffWorldclim[Colombia-Belize].zip', 'r') as temp:
  temp.extractall("rasters")

# path to the folder
files = glob.glob(os.path.join('/content/rasters', '*tif'))

###### list of tiff files ######
# Rename and arrange the information
names = [i[i.rfind('/')+1:-4] for i in files]

# Separate datasets (this is pretty redundant)
belize, colombia = [i for i in names if i.startswith('Belize')], [i for i in names if i.startswith('Colombia')]

# It will work only on 2.5s and 30s maps(Columbia 15, Belize 13)
Belize, Colombia = [i for i in belize if i[14:16] in ['30', '2_']], [i for i in colombia if i[16:18] in ['30', '2_']]

In [40]:
##################### Rasters temperature ######################################
# reading the data
path, ext = '/content/rasters/', '.tif'

# importing and handling
amt_belize = []
for i in Belize:
  if i.endswith('BIO2'):
    name = i
    ds = rasterio.open(path + i + ext, 'r')
    data = ds.read()
    max = np.nanmax(data)
    min = np.nanmin(data)
    amt_belize.append((name, ds, data, min, max))
  else:
    next

amt_colombia = []
for i in Colombia:
  if i.endswith('BIO2'):
    name = i
    ds = rasterio.open(path + i + ext, 'r')
    data = ds.read()
    max = np.nanmax(data)
    min = np.nanmin(data)
    amt_colombia.append((name, ds, data, min, max))
  else:
    next

amt_belize.sort()
amt_colombia.sort()

In [48]:
##################### Rasters temperature ######################################
# reading the data
path, ext = '/content/rasters/', '.tif'

# importing and handling
tpa_belize = []
for i in Belize:
  if i.endswith('BIO12'):
    name = i
    ds = rasterio.open(path + i + ext, 'r')
    data = ds.read()
    max = np.nanmax(data)
    min = np.nanmin(data)
    tpa_belize.append((name, ds, data, min, max))
  else:
    next

tpa_colombia = []
for i in Colombia:
  if i.endswith('BIO12'):
    name = i
    ds = rasterio.open(path + i + ext, 'r')
    data = ds.read()
    max = np.nanmax(data)
    min = np.nanmin(data)
    tpa_colombia.append((name, ds, data, min, max))
  else:
    next

tpa_belize.sort()
tpa_colombia.sort()

In [53]:
# removing zip folder
%rm "clip_tiffWorldclim[Colombia-Belize].zip"
%rm -r "rasters"

# **Life Zones Reported** - value to reclassify

- **Precipitación**\
  1) Menos de 1000mm\
  2) 1000mm - 2000mm\
  3) 2000mm - 4000mm\
  4) 4000mm - 8000mm\
  5) Mas de 8000m \

- **Temperatura**\
  6) Menos de 6°C\
  7) 6°C - 12°C\
  8) 12°C - 18°C\
  9) 18°C - 24°C\
  10 Mas de 24°C \

- **Altitud (msnm)**\
  11) 0 - 1000\
  12) 1000 - 2000\
  13) 2000 - 3000\
  14) 3000 - 4000

## **Biotemperature** (geeting by product Temperature x Altitude)

- **91:** 2000 - 3000 m / 6 - 12°C (Montane Subtropical)
- **96:** 1000 - 2000 m / 112 - 18 °C (Lower Montane Subtropical)
- **98:** 3000 - 4000 m / 6 -12°C (Montane Tropical)
- **99:** 0 - 1000 m / 18 - 24°C (Premontane Tropical)
- **104:** 2000 - 3000m / 12 - 18°C (Lower montane tropical)
- **108:** 1000 - 2000m / 18 - 24°C (Premontane tropical)
- **110:** 0 - 1000m / mas de 24 (Base Tropical)


# **1. Colombia**

In [59]:
# REVIEW VALUES MAX AND MIN 
for i in tpa_colombia:
  print(f'{i[0]}\t with max values: {i[3]:.2f} and min values: {i[4]:.2f}')
print()
for i in amt_colombia:
  print(f'{i[0]}\t with max values: {i[3]:.2f} and min values: {i[4]:.2f}')
print()
for i in elev_colombia:
  print(f'{i[0]}\t with max values: {i[3]:.2f} and min values: {i[4]:.2f}')

Colombia-BioClim2_5_BIO12	 with max values: 344.00 and min values: 7312.00
Colombia-BioClim30s_BIO12	 with max values: 283.00 and min values: 8029.00

Colombia-BioClim2_5_BIO2	 with max values: 6.04 and min values: 12.77
Colombia-BioClim30s_BIO2	 with max values: 5.72 and min values: 13.11

Colombia_WC2.1_2.5M_ELEV	 with max values: -46.00 and min values: 4834.00
Colombia_WC2.1_30S_ELEV	 with max values: -65.00 and min values: 5463.00


## **2.5 - Colombia**

In [61]:
# reclassifiying tpa
print(tpa_colombia[0][0])
tpaCol_2_5 = tpa_colombia[0][2].copy()
tpaCol_2_5[np.where((tpaCol_2_5 <= 1000))] = 1
tpaCol_2_5[np.where((tpaCol_2_5 >= 1000) & (tpaCol_2_5 <=2000))] = 2
tpaCol_2_5[np.where((tpaCol_2_5 >= 2000) & (tpaCol_2_5 <=4000))] = 3
tpaCol_2_5[np.where((tpaCol_2_5 >= 4000) & (tpaCol_2_5 <=8000))] = 4
# dont have 5

Colombia-BioClim2_5_BIO12


In [62]:
# reclasifying amt
print(amt_colombia[0][0])
amtCol_2_5 = amt_colombia[0][2].copy()
# dont have 6
amtCol_2_5[np.where((amtCol_2_5 >= 6) & (amtCol_2_5 <=12))] = 7
amtCol_2_5[np.where((amtCol_2_5 >= 12) & (amtCol_2_5 <=18))] = 8
amtCol_2_5[np.where((amtCol_2_5 >= 18) & (amtCol_2_5 <=24))] = 9
# dont have 10

Colombia-BioClim2_5_BIO2


In [ ]:
# reclasifying elev
print(elev_colombia[0][0])
elevCol_2_5 = elev_colombia[0][2].copy()
elevCol_2_5[np.where((elevCol_2_5 <= 1000))] = 11
elevCol_2_5[np.where((elevCol_2_5 >= 1000) & (elevCol_2_5 <= 2000))] = 12
elevCol_2_5[np.where((elevCol_2_5 >= 2000) & (elevCol_2_5 <= 3000))] = 13
elevCol_2_5[np.where((elevCol_2_5 >= 3000) & (elevCol_2_5 <= 4000))] = 14
elevCol_2_5[np.where((elevCol_2_5 <= 4000))] = 15

In [16]:
np.nanmax(lista)

4.0

- **Precipitation**\
  1) Menos de 1000 mm\
  2) 1000 mm - 2000 mm\
  3) 2000 mm - 4000 mm\
  4) 4000 mm - 8000 mm\
  5) Mas de 8000 mm\

- **Temperature**\
  6) Menos de 6°C\
  7) 6 °C - 12 °C\
  8) 12 °C - 18 °C\
  9) 18 °C - 24 °C\
  10) Mas de 24 °C\

- **Altitud (msnm)**\
  11) 0 - 1000\
  12) 1000 - 2000\
  13) 2000 - 3000\
  14) 3000 - 4000\
  

# **2. Belize**

In [17]:
for i in belize:
  print(f'{i[0]}\t with max values: {i[3]:.2f} and min values: {i[4]:.2f}')

Belize-BioClim2_5_BIO12	 with max values: 1185.00 and min values: 3605.00
Belize-BioClim2_5_BIO2	 with max values: 7.49 and min values: 13.01
Belize-BioClim30s_BIO12	 with max values: 1181.00 and min values: 3619.00
Belize-BioClim30s_BIO2	 with max values: 7.38 and min values: 13.03


In [18]:
# getting array
array_belize = []
for i in belize:
  name, geotiff = i
  array = geotiff.read()
  array_belize.append((name, geotiff, array))

ValueError: ignored

In [ ]:
for i in range(1,len(tiffs)):
  fig, (ax1, ax2) = plt.subplots(1,2, figsize=(10, 8))
  prueba = tiffs[i]
  show(prueba[1], ax=ax1, title = prueba[0])
  show_hist(prueba[1], title='Histogram for ' + prueba[0], ax=ax2)
  plt.show()

In [ ]:
prueba[1].meta

In [ ]:
prueba[1].nodata

In [ ]:
prueba[1].transform

In [ ]:
# Assign raster values to nmpy nd arrray
prueba2 = prueba[1].read()
prueba2

# prueba con gdal

In [ ]:
gtiffs = list(zip(names, [gdal.Open(f) for f in files]))
gtiffs.sort(key=lambda x:x[0])
gtiffs

In [ ]:
files[1] == files[2]

In [ ]:
prueba = gtiffs[4][1]
print(gtiffs[4][0], ' Temperature')
# print(gtiffs[0][0], ' Precipitation')

# **Geotransform**
A geotransform is an affine transformation from the image coordinate space (row, column), also known as (pixel, line) to the georeferenced coordinate space (projected or geographic coordinates).\
**GT(0)** - x-coordinate of the upper-left corner of the upper-left pixel.\
**GT(1)** - w-e pixel resolution / pixel width.\
**GT(2)** - row rotation (typically zero).\
**GT(3)** - y-coordinate of the upper-left corner of the upper-left pixel.\
**GT(4)** - column rotation (typically zero).\
**GT(5)** - n-s pixel resolution / pixel height (negative value for a north-up image.\

In [ ]:
# get transform ()
gt = prueba.GetGeoTransform()
gt

In [ ]:
# get projection
proj = prueba.GetProjection()
proj

In [ ]:
# get raster band
band = prueba.GetRasterBand(1)
band

In [ ]:
# create an array from arrays
array= band.ReadAsArray()
array

In [ ]:
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt

# import 
#ds = gdal.Open()
ds = prueba
gt = ds.GetGeoTransform()
proj = ds.GetProjection()

band = ds.GetRasterBand(1)
array = band.ReadAsArray()

plt.figure()
plt.imshow(array)
plt.show()

In [ ]:
# manipulate
binmask = np.where((array >= np.mean(array)),1,0)
plt.figure()
plt.imshow(binmask)
plt.show()

# export
driver = gdal.GetDriverByName("GTiff")
driver.Register()
outds = driver.Create("binmask.tif", xsize = binmask.shape[1],
                      ysize = binmask.shape[0], bands = 1, 
                      eType = gdal.GDT_Int16)
outds.SetGeoTransform(gt)
outds.SetProjection(proj)
outband = outds.GetRasterBand(1)
outband.WriteArray(binmask)
outband.SetNoDataValue(np.nan)
outband.FlushCache()

# close your datasets and bands!!!
outband = None
outds = None